# Short Enwiki Train

Test that the model init code, runs without issues

**L6-D512 model with**
- Layer count: 6
- Embed size: 512

## Preparing the init model and test dataset

In [1]:
GPU_DEVICES="auto"
ENABLE_WANDB=False
WANDB_PREFIX="infctx-v5-unit-test"
DEEPSPEED_STRAT="deepspeed_stage_1"

print("ENABLE_WANDB:", ENABLE_WANDB)
print("GPU_DEVICES:", GPU_DEVICES)

if ENABLE_WANDB:
    WANDB_MODE="online"
else:
    WANDB_MODE="disabled"

# Computing the notebook, and various paths
import os
NOTEBOOK_DIR=os.path.dirname(os.path.abspath("__file__"))
PROJECT_DIR=os.path.abspath(os.path.join(NOTEBOOK_DIR, "../../"))
TRAINER_DIR=os.path.abspath(os.path.join(PROJECT_DIR, "./RWKV-v5/"))

print("NOTEBOOK_DIR:", NOTEBOOK_DIR)
print("TRAINER_DIR:", TRAINER_DIR)
print("PROJECT_DIR:", PROJECT_DIR)

ENABLE_WANDB: False
GPU_DEVICES: auto
NOTEBOOK_DIR: /home/recursal/RWKV-infctx-trainer/notebook/trainer-v5-unit-test
TRAINER_DIR: /home/recursal/RWKV-infctx-trainer/RWKV-v5
PROJECT_DIR: /home/recursal/RWKV-infctx-trainer


In [10]:
# First lets setup the various directories
!mkdir -p "{PROJECT_DIR}/model/"
!mkdir -p "{PROJECT_DIR}/datapath/"
!mkdir -p "{PROJECT_DIR}/checkpoint/"

# Initialize the model and dataset

In [11]:
# Lets initialized the L6-D512 model with the init_model.py code
!cd "{TRAINER_DIR}" && python3 init_model.py \
    --n_layer 6 --n_embd 512 \
    --vocab_size world \
    --skip-if-exists --safe-init \
    ../model/L6-D512-world-init.pth

[2024-05-03 12:31:50,436] [INFO] [real_accelerator.py:161:get_accelerator] Setting ds_accelerator to cuda (auto detect)
[RWKV.model] Running RWKV infctx using 'torch-jit' with torch '2.1.2'
---- Initializing model ----
No of layers: 6
Embedding size: 512
Output model path: ../model/L6-D512-world-init.pth
Vocab size: 65536
Emb scale: 0.0001
Note: this process takes a significant time (and ram) for large models
---- ----- ----
Output model exists, skipping init_model


In [12]:
# Preload the dataset
!cd "{TRAINER_DIR}" && \
    python3 preload_datapath.py "{NOTEBOOK_DIR}/config/enwiki_10k-world-4x1024.yaml"

Saving the dataset (1/1 shards): 100%|█| 690/690 [00:00<00:00, 3490.58 examples/
Saving the dataset (1/1 shards): 100%|███| 7/7 [00:00<00:00, 2845.80 examples/s]


# Perform the initial test run

In [13]:
# Short training process - for quick testing / debugging
!cd "{TRAINER_DIR}" && \
    export WANDB_MODE="disabled" && \
    python3 lightning_trainer.py fit \
        -c "{NOTEBOOK_DIR}/config/enwiki_10k-world-4x1024.yaml" \
        --trainer.logger.init_args.name="{WANDB_PREFIX} (train-ctx=1024, data-ctx=4096, {DEEPSPEED_STRAT})" \
        --trainer.strategy="{DEEPSPEED_STRAT}" \
        --trainer.devices="{GPU_DEVICES}" \
        --trainer.fast_dev_run=2 \
        --model.load_model="../model/L6-D512-world-init.pth"

[2024-05-03 12:32:03,660] [INFO] [real_accelerator.py:161:get_accelerator] Setting ds_accelerator to cuda (auto detect)
[RWKV.model] Running RWKV infctx using 'torch-jit' with torch '2.1.2'
/home/recursal/miniconda3/envs/rwkv-infctx/lib/python3.11/site-packages/lightning/pytorch/cli.py:518: LightningCLI's args parameter is intended to run from within Python like if it were from the command line. To prevent mistakes it is not recommended to provide both args and command line arguments, got: sys.argv[1:]=['fit', '-c', '/home/recursal/RWKV-infctx-trainer/notebook/trainer-v5-unit-test/config/enwiki_10k-world-4x1024.yaml', '--trainer.logger.init_args.name=infctx-v5-unit-test (train-ctx=1024, data-ctx=4096, deepspeed_stage_1)', '--trainer.strategy=deepspeed_stage_1', '--trainer.devices=auto', '--trainer.fast_dev_run=2', '--model.load_model=../model/L6-D512-world-init.pth'], args=['fit', '-c', '/home/recursal/RWKV-infctx-trainer/notebook/trainer-v5-unit-test/config/enwiki_10k-world-4x1024.yam

# Reset checkpoint dir, and perform a slightly longer run

In [ ]:
# Empty out the checkpoint
!cd "{PROJECT_DIR}" && rm -rf "./checkpoint/infctx-v5-unit-test-baseline-4x1024/"

In [15]:
# Longer training process
!cd "{TRAINER_DIR}" && \
    export WANDB_MODE="{WANDB_MODE}" && \
    python3 lightning_trainer.py fit \
        -c "{NOTEBOOK_DIR}/config/enwiki_10k-world-4x1024.yaml" \
        --trainer.logger.init_args.name="{WANDB_PREFIX} (train-ctx=1024, data-ctx=4096, {DEEPSPEED_STRAT})" \
        --trainer.strategy="{DEEPSPEED_STRAT}" \
        --trainer.devices="{GPU_DEVICES}" \
        --trainer.microbatch_size=8 \
        --model.load_model="../model/L6-D512-world-init.pth"
        

[2024-05-03 12:32:31,433] [INFO] [real_accelerator.py:161:get_accelerator] Setting ds_accelerator to cuda (auto detect)
[RWKV.model] Running RWKV infctx using 'torch-jit' with torch '2.1.2'
/home/recursal/miniconda3/envs/rwkv-infctx/lib/python3.11/site-packages/lightning/pytorch/cli.py:518: LightningCLI's args parameter is intended to run from within Python like if it were from the command line. To prevent mistakes it is not recommended to provide both args and command line arguments, got: sys.argv[1:]=['fit', '-c', '/home/recursal/RWKV-infctx-trainer/notebook/trainer-v5-unit-test/config/enwiki_10k-world-4x1024.yaml', '--trainer.logger.init_args.name=infctx-v5-unit-test (train-ctx=1024, data-ctx=4096, deepspeed_stage_1)', '--trainer.strategy=deepspeed_stage_1', '--trainer.devices=auto', '--trainer.microbatch_size=8', '--model.load_model=../model/L6-D512-world-init.pth'], args=['fit', '-c', '/home/recursal/RWKV-infctx-trainer/notebook/trainer-v5-unit-test/config/enwiki_10k-world-4x1024.

# Export and test the model

In [16]:
# Lets export the model from the checkpoint
!cd "{TRAINER_DIR}" && \
    python export_checkpoint.py "../checkpoint/infctx-v5-unit-test-baseline-4x1024/last.ckpt" "../model/infctx-v5-unit-test-baseline-4x1024.pth"
!cd "{TRAINER_DIR}" && ls -alh "../model/infctx-v5-unit-test-baseline-4x1024.pth"

[2024-05-03 12:33:13,067] [INFO] [real_accelerator.py:161:get_accelerator] Setting ds_accelerator to cuda (auto detect)
Processing zero checkpoint '../checkpoint/infctx-v5-unit-test-baseline-4x1024/last.ckpt/checkpoint'
Detected checkpoint of type zero stage ZeroStageEnum.optimizer_states, world_size: 8
Parsing checkpoint created by deepspeed==0.12.6
Reconstructed fp32 state dict with 138 params 87601152 elements
Saving bf16 state dict to ../model/infctx-v5-unit-test-baseline-4x1024.pth
-rw-rw-r-- 1 recursal recursal 168M May  3 12:33 ../model/infctx-v5-unit-test-baseline-4x1024.pth


In [4]:
# Lets do a quick dragon prompt validation
!cd "{TRAINER_DIR}" && \
    python3 dragon_test.py "../model/infctx-v5-unit-test-baseline-4x1024.pth" "cuda fp32" 10

[2024-05-04 09:03:19,331] [INFO] [real_accelerator.py:161:get_accelerator] Setting ds_accelerator to cuda (auto detect)
[RWKV.model] Running RWKV infctx using 'torch-jit' with torch '2.1.2'
[SimpleRWKV] Warning: dtype mismatch, only fp32 is supported (for now)
/home/recursal/miniconda3/envs/rwkv-infctx/lib/python3.11/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
---
[RWKV.TimeMix] Compiling CUDA kernel with HEAD_SIZE=64
Using /home/recursal/.cache/torch_extensions/py311_cu121 as PyTorch extensions root...
Detected CUDA files, patching ldflags
Emitting ninja build file /home/recursal/.cache/torch_extensions/py311_cu121/wkv5/build.ninja...
Building extension module wkv5...


# Perform state tuning

In [16]:
# Empty out the checkpoint
!cd "{PROJECT_DIR}" && rm -rf "./checkpoint/infctx-v5-unit-test-baseline-4x1024/"

In [17]:
# Longer training process
!cd "{TRAINER_DIR}" && \
    export WANDB_MODE="{WANDB_MODE}" && \
    python3 lightning_trainer.py fit \
        -c "{NOTEBOOK_DIR}/config/enwiki_10k-world-4x1024.yaml" \
        --trainer.logger.init_args.name="{WANDB_PREFIX} (train-ctx=1024, data-ctx=4096, {DEEPSPEED_STRAT})" \
        --trainer.strategy="{DEEPSPEED_STRAT}" \
        --trainer.devices="{GPU_DEVICES}" \
        --trainer.microbatch_size=8 \
        --model.state_tuning=True \
        --model.load_model="../model/infctx-v5-unit-test-baseline-4x1024.pth"
        

[2024-05-04 09:28:07,126] [INFO] [real_accelerator.py:161:get_accelerator] Setting ds_accelerator to cuda (auto detect)
[RWKV.model] Running RWKV infctx using 'torch-jit' with torch '2.1.2'
/home/recursal/miniconda3/envs/rwkv-infctx/lib/python3.11/site-packages/lightning/pytorch/cli.py:518: LightningCLI's args parameter is intended to run from within Python like if it were from the command line. To prevent mistakes it is not recommended to provide both args and command line arguments, got: sys.argv[1:]=['fit', '-c', '/home/recursal/RWKV-infctx-trainer/notebook/trainer-v5-unit-test/config/enwiki_10k-world-4x1024.yaml', '--trainer.logger.init_args.name=infctx-v5-unit-test (train-ctx=1024, data-ctx=4096, deepspeed_stage_1)', '--trainer.strategy=deepspeed_stage_1', '--trainer.devices=auto', '--trainer.microbatch_size=8', '--model.state_tuning=True', '--model.load_model=../model/infctx-v5-unit-test-baseline-4x1024.pth'], args=['fit', '-c', '/home/recursal/RWKV-infctx-trainer/notebook/traine

In [20]:
# Lets export the model from the checkpoint
!cd "{TRAINER_DIR}" && \
    python export_checkpoint.py "../checkpoint/infctx-v5-unit-test-baseline-4x1024/last.ckpt" "../model/infctx-v5-unit-test-baseline-4x1024-statetune.pth"
!cd "{TRAINER_DIR}" && ls -alh "../model/infctx-v5-unit-test-baseline-4x1024-statetune.pth"

[2024-05-04 09:29:54,456] [INFO] [real_accelerator.py:161:get_accelerator] Setting ds_accelerator to cuda (auto detect)
Processing zero checkpoint '../checkpoint/infctx-v5-unit-test-baseline-4x1024/last.ckpt/checkpoint'
Detected checkpoint of type zero stage ZeroStageEnum.optimizer_states, world_size: 8
Parsing checkpoint created by deepspeed==0.12.6
Reconstructed fp32 state dict with 139 params 87797760 elements
Saving bf16 state dict to ../model/infctx-v5-unit-test-baseline-4x1024-statetune.pth
-rw-rw-r-- 1 recursal recursal 168M May  4 09:29 ../model/infctx-v5-unit-test-baseline-4x1024-statetune.pth


In [21]:
# Lets do a quick dragon prompt validation
!cd "{TRAINER_DIR}" && \
    python3 dragon_test.py "../model/infctx-v5-unit-test-baseline-4x1024-statetune.pth" "cuda fp32" 10

[2024-05-04 09:30:01,454] [INFO] [real_accelerator.py:161:get_accelerator] Setting ds_accelerator to cuda (auto detect)
[RWKV.model] Running RWKV infctx using 'torch-jit' with torch '2.1.2'
[SimpleRWKV] Warning: dtype mismatch, only fp32 is supported (for now)
/home/recursal/miniconda3/envs/rwkv-infctx/lib/python3.11/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
---
[RWKV.TimeMix] Compiling CUDA kernel with HEAD_SIZE=64
Using /home/recursal/.cache/torch_extensions/py311_cu121 as PyTorch extensions root...
Detected CUDA files, patching ldflags
Emitting ninja build file /home/recursal/.cache/torch_extensions/py311_cu121/wkv5/build.ninja...
Building extension module wkv5...
